In [19]:
import pandas as pd
import os
import fnmatch
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

def get_subdirectories_with_measurements(directory: str):
    matches = []
    for root, dirs, files in os.walk(directory):
        for subdir in dirs:
            if fnmatch.fnmatch(subdir, 'measurements_sleep*'):
                matches.append(os.path.join(root, subdir))
    return matches

def create_graphs_for_experiment(path):
    measurements = pd.read_csv(f"{path}/measurements.csv", decimal='.')

    timings = measurements.tail(2)
    timings.columns = ['timestamp']

    start = timings['timestamp'].min()
    end = timings['timestamp'].max()
    runtime = end - start

    # Omit the last two rows
    power_draws = measurements.iloc[:-2]
    power_draws.columns = ['current']

    time_between_measurements = runtime / power_draws.shape[0]

    power_draws['time'] = power_draws.apply(lambda row: time_between_measurements * row.name, axis=1)
    power_draws['timestamp'] = power_draws.apply(lambda row: start + time_between_measurements * row.name, axis=1)

    fig = px.line(power_draws, x='time', y='current')

    events = pd.read_csv(f"{path}/events.csv", header=None) if os.path.exists(f"{path}/events.csv") else pd.DataFrame()
    if (not events.empty):
        events.columns = ['timestamp', 'label']
    
        timestamps_for_this_run = events # [(fmnist_timestamps['timestamp'] > start) & (fmnist_timestamps['timestamp'] < end)]
        timestamps_for_this_run['time'] = timestamps_for_this_run['timestamp'].apply(lambda timestamp: timestamp - start)
        for index, row in timestamps_for_this_run.iterrows():
            fig.add_vline(x=row['time'], line_width=3, line_color="green", annotation=dict(text=row['label'], textangle=-90))

    fig.add_hline(y=55*1000, line_dash="dot",
                annotation_text="baseline", 
                annotation_position="bottom right")



    filepath = f"{path}/plot.png"
    print(f"Saving to {filepath}")
    pio.write_image(fig, file=filepath, format='png', width=1920, height=1080)

# for path in get_subdirectories_with_measurements('.'):
#     print(f'Plotting {path}')
#     create_graphs_for_experiment(path)

Plotting ./measurements_sleep_0708212337
Saving to ./measurements_sleep_0708212337/plot.png


/tmp/ipykernel_37275/998100046.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_37275/998100046.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Plotting ./measurements_sleep_0708210826
Saving to ./measurements_sleep_0708210826/plot.png


/tmp/ipykernel_37275/998100046.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_37275/998100046.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Plotting ./measurements_sleep_0708205715
Saving to ./measurements_sleep_0708205715/plot.png


...

/tmp/ipykernel_37275/998100046.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_37275/998100046.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Plotting ./measurements_sleep_0708210024
Saving to ./measurements_sleep_0708210024/plot.png


/tmp/ipykernel_37275/998100046.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_37275/998100046.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Plotting ./measurements_sleep_0708210616
Saving to ./measurements_sleep_0708210616/plot.png


/tmp/ipykernel_37275/998100046.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_37275/998100046.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

